# Imports

In [ ]:
import pandas as pd
from asapdiscovery.data.schema.ligand import Ligand
from asapdiscovery.data.readers.molfile import MolFileFactory
from rdkit.Chem import Draw, rdMolAlign, rdDepictor
from rdkit import Chem

from rdkit.Chem.Scaffolds import MurckoScaffold
from collections import defaultdict

# Load the Ligands

In [ ]:
mff = MolFileFactory(filename="combined_3d.sdf")
ligs = mff.load()

In [ ]:
x_series_ligs = [lig for lig in ligs if "x" in lig.tags["xtal_name"]]

In [ ]:
len(x_series_ligs)

In [ ]:
lig_name = {lig.compound_name: lig for lig in x_series_ligs}

In [ ]:
len(lig_name)

In [ ]:
lig_dict = defaultdict(list)
for lig in x_series_ligs:
    lig_dict[lig.compound_name].append(lig)

In [ ]:
doubled_lig_dict = {name: [lig.tags["xtal_name"] for lig in lig_list] for name, lig_list in lig_dict.items() if len(lig_list) > 1}

In [ ]:
doubled_lig_dict

# Function Definitions

In [ ]:
def generate_scaffold(ligand: Ligand, include_chirality=True):
    """
    Compute the Bemis-Murcko scaffold for a SMILES string.
    Implementation copied from https://github.com/chemprop/chemprop.

    :param mol: A smiles string or an RDKit molecule.
    :param include_chirality: Whether to include chirality.
    :return:
    """
    mol = ligand.to_rdkit()
    scaffold = MurckoScaffold.MurckoScaffoldSmiles(mol=mol, includeChirality=include_chirality)
    
    # scaffold = Chem.MolToSmiles(MurckoScaffold.MakeScaffoldGeneric(mol))

    return scaffold

In [ ]:
def split_by_scaffold(ligands):
    """
    Split ligands by scaffold.
    """
    
    scaffolds = defaultdict(list)
    for ligand in ligands:
        scaffold = generate_scaffold(ligand)
        scaffolds[scaffold].append(ligand)
    scaffold_list = [{"scaffold": scaffold, "ligands": ligands} for scaffold, ligands in scaffolds.items()]
    return sorted(scaffold_list, key=lambda x: len(x["ligands"]), reverse=True)

In [ ]:
def align_to_scaffold(scaffold: Chem.Mol, mols: list[Chem.Mol]):
    rdDepictor.Compute2DCoords(scaffold)
    template_match = scaffold.GetSubstructMatch(scaffold)
    for mol in mols:
        rdDepictor.Compute2DCoords(mol)
        query_match = mol.GetSubstructMatch(scaffold)
        rdMolAlign.AlignMol(mol, scaffold, atomMap=list(zip(query_match, template_match)))
    return scaffold, mols

In [ ]:
def draw_scaffold_cluster(scaffold: str, ligands: list[Ligand], filename: str):
    
    # Convert to rdkit
    rdkitmols = [Chem.RemoveHs(ligand.to_rdkit()) for ligand in ligands]
    rdkit_scaffold = Chem.MolFromSmiles(scaffold)
    
    # Align the molecules to the scaffold
    rdkit_scaffold, rdkitmols = align_to_scaffold(rdkit_scaffold, rdkitmols)
    
    
    # Find the atoms to highlight
    highlight = [mol.GetSubstructMatch(rdkit_scaffold) for mol in rdkitmols]
    
    # Set Draw Options
    dopts = Draw.rdMolDraw2D.MolDrawOptions()
    dopts.setHighlightColour((68/256, 178/256, 212/256))
    dopts.highlightBondWidthMultiplier = 16
    
    # Draw the molecules
    img = Draw.MolsToGridImage(rdkitmols, 
                               molsPerRow=6, 
                               subImgSize=(200, 200), 
                               highlightAtomLists=highlight,
                               useSVG= True,
                               legends = [f"{ligand.compound_name} ({ligand.tags['xtal_name']})" for ligand in ligands],
                               drawOptions=dopts)
    with open(filename, 'w') as f:
        f.write(img.data)

# Execution

In [ ]:
scaffolds = split_by_scaffold(x_series_ligs)

In [ ]:
cluster_labels = []
for i, scaffold_dict in enumerate(scaffolds): 
    try:
        draw_scaffold_cluster(scaffold_dict['scaffold'], scaffold_dict['ligands'], f"scaffold_{i}.svg")
    except Exception as e:
        print(f"Failed to draw scaffold {i}")
        print(e)
    for ligand in scaffold_dict['ligands']:
        cluster_labels.append({"Name": ligand.compound_name, "Cluster": i, "Scaffold_Smiles": scaffold_dict['scaffold']})

# Save Label Dict

In [ ]:
import pandas as pd

In [ ]:
cluster_df = pd.DataFrame.from_records(cluster_labels)

In [ ]:
cluster_df.to_csv("cluster_labels.csv", index=False)

In [ ]:
cluster_df.groupby("Cluster").count()

# Draw Scaffolds

In [ ]:
scaffold_smiles = [Ligand.from_smiles(scaffold_dict['scaffold'], compound_name=f"Cluster {i} - {len(scaffold_dict['ligands'])} molecules") for i, scaffold_dict in enumerate(scaffolds)]

In [ ]:
scaffold_rdmols = [Chem.RemoveHs(ligand.to_rdkit()) for ligand in scaffold_smiles]

In [ ]:
n_scaffolds = 12

In [ ]:
# Set Draw Options
dopts = Draw.rdMolDraw2D.MolDrawOptions()
dopts.setHighlightColour((68/256, 178/256, 212/256))
dopts.highlightBondWidthMultiplier = 16

In [ ]:
from rdkit.Chem import Draw, rdDepictor
d2d = Draw.MolDraw2DCairo(350,300)
for mol in scaffold_rdmols[:n_scaffolds]:
    Draw.MolToImage(mol, size=(200, 200), options=dopts)
    rdDepictor.Compute2DCoords(mol)
    rdDepictor.StraightenDepiction(mol)
    d2d.DrawMolecule(mol)

In [ ]:
img = Draw.MolsToGridImage(scaffold_rdmols[:n_scaffolds], 
                               molsPerRow=3, 
                               subImgSize=(200, 200), 
                               # highlightAtomLists=highlight,
                               useSVG= True,
                               legends = [ligand.compound_name for ligand in scaffold_smiles[:n_scaffolds]],
                               drawOptions=dopts)
with open("scaffold_images_3x4.svg", 'w') as f:
        f.write(img.data)

In [ ]:
img = Draw.MolsToGridImage(scaffold_rdmols[:12], 
                               molsPerRow=6, 
                               subImgSize=(200, 200), 
                               # highlightAtomLists=highlight,
                               useSVG= True,
                               legends = [ligand.compound_name for ligand in scaffold_smiles[:12]],
                               drawOptions=dopts)
with open("scaffold_images_6x2.svg", 'w') as f:
        f.write(img.data)